In [1]:
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from pandas import ExcelWriter
from pandas import ExcelFile
 
df1 = pd.read_excel(r'/Users/varun/Desktop/ST/data_in.xlsx', sheetname='Sheet1')
print("The shape of our data:",df1.shape,"\n") 
print("Column headings:")
print(df1.columns)

The shape of our data: (401, 8) 

Column headings:
Index(['Work Card Ref', 'ATA ', 'Discrepancy / Action Required',
       'Rectification', 'Year', 'Unnamed: 5', 'Defect Type', 'Defect Item'],
      dtype='object')


/Users/varun/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [3]:
df1.drop(['Work Card Ref', 'Year'], axis=1)

,ATA,Discrepancy / Action Required,Rectification,Unnamed: 5,Defect Type,Defect Item
0,25,DEFIBRILLATOR BATTERY - INSPECT/CHECK.,CARRIED OUT DEFIBRILLATOR BATTERY INSPECTION C...,NaN,NaN,NaN
1,25,DEFIBRILLATOR ELECTRODE PADS INSPECTION.,CARRIED OUT DEFIBRILLATOR ELECTRODE PADS INSPE...,NaN,NaN,NaN
2,32,RH NOSE WHEEL TIRE FOUND WORN,RH NOSE WHEEL ASSEMBLY REMOVED AND SERVICEABLE...,NaN,WORN,TIRE
3,24,TAIL COMPARTMENT GROUND SERVICE BUS INDICATOR ...,TAIL COMPARTMENT GROUND SERVICE BUS INDICATOR ...,NaN,ILLUMINATED PERMANENTLY,INDICATOR
4,34,VOR - IFR CHECK,CARRIED OUT VOR - IFR CHECK IN ACCORDANCE WITH...,NaN,NaN,NaN
5,30,"PILOT REPORTS ""RH ICE DETECTOR AMBER MESSAGE A...",RH ICE DETECTOR PROBE REPLACED IN ACCORDANCE W...,NaN,MESSAGE ALWAYS POSTED,ICE DETECTOR MESSAGE
6,34,TO CARRY OUT HONEYWELL ASB ADM/ADC/ADAHRS-34-AD1,NOT APPLICABLE - NOT AFFECTED BY SERIAL NO. IN...,NaN,NaN,NaN
7,25,ESPRESSO MAUER FOUND UNSERVICEABLE,DEFECTIVE ESPRESSO MAUER REMOVED AND SERVICEAB...,NaN,UNSERVICEABLE,ESPRESSO MAUER
8,32,NOSE WHEEL ASSEMBLY TO BE BUILT UP.\nMEGGITT M...,"NOSE WHEEL ASSEMBLY DISASSEMBLED, CLEANED, INS...",NaN,NaN,NaN
9,49,FUEL FILTER BYPASS INDICATOR DETAILED INSPECTI...,FUEL FILTER BYPASS INDICATOR DETAILED INSPECTI...,NaN,NaN,NaN


In [4]:
df1 = df1.rename(columns={'Discrepancy / Action Required': 'Problem'})
df1["Defect Type"]=df1["Defect Type"].astype(str)
df1["Defect Item"]=df1["Defect Item"].astype(str)

In [5]:
df1['Problem_words'] = df1.Problem.str.strip().str.split('[\W_]+')
df1['Rectification_words'] = df1.Rectification.str.strip().str.split('[\W_]+')
#df1.drop(["words"], axis = 1, inplace = True) 
df1.head()

,Work Card Ref,ATA,Problem,Rectification,Year,Unnamed: 5,Defect Type,Defect Item,Problem_words,Rectification_words
0,256404,25,DEFIBRILLATOR BATTERY - INSPECT/CHECK.,CARRIED OUT DEFIBRILLATOR BATTERY INSPECTION C...,2013,NaN,nan,nan,"[DEFIBRILLATOR, BATTERY, INSPECT, CHECK, ]","[CARRIED, OUT, DEFIBRILLATOR, BATTERY, INSPECT..."
1,256407,25,DEFIBRILLATOR ELECTRODE PADS INSPECTION.,CARRIED OUT DEFIBRILLATOR ELECTRODE PADS INSPE...,2013,NaN,nan,nan,"[DEFIBRILLATOR, ELECTRODE, PADS, INSPECTION, ]","[CARRIED, OUT, DEFIBRILLATOR, ELECTRODE, PADS,..."
2,"324002, 324008, 324014, 324041",32,RH NOSE WHEEL TIRE FOUND WORN,RH NOSE WHEEL ASSEMBLY REMOVED AND SERVICEABLE...,2013,NaN,WORN,TIRE,"[RH, NOSE, WHEEL, TIRE, FOUND, WORN]","[RH, NOSE, WHEEL, ASSEMBLY, REMOVED, AND, SERV..."
3,NaN,24,TAIL COMPARTMENT GROUND SERVICE BUS INDICATOR ...,TAIL COMPARTMENT GROUND SERVICE BUS INDICATOR ...,2013,NaN,ILLUMINATED PERMANENTLY,INDICATOR,"[TAIL, COMPARTMENT, GROUND, SERVICE, BUS, INDI...","[TAIL, COMPARTMENT, GROUND, SERVICE, BUS, INDI..."
4,345004,34,VOR - IFR CHECK,CARRIED OUT VOR - IFR CHECK IN ACCORDANCE WITH...,2013,NaN,nan,nan,"[VOR, IFR, CHECK]","[CARRIED, OUT, VOR, IFR, CHECK, IN, ACCORDANCE..."


In [6]:
df1['Rectification_words'][0]

['CARRIED',
 'OUT',
 'DEFIBRILLATOR',
 'BATTERY',
 'INSPECTION',
 'CHECK',
 'IN',
 'ACCORDANCE',
 'WITH',
 'OPERATIONAL',
 'MANUAL',
 'AND',
 'FOUND',
 'SATISFACTORY',
 '']

In [7]:
df1.isnull().sum()
#df1["Problem_words"]=df1["Problem_words"].astype(str)

Work Card Ref          127
ATA                      3
Problem                  0
Rectification            0
Year                     0
Unnamed: 5             401
Defect Type              0
Defect Item              0
Problem_words            0
Rectification_words      0
dtype: int64

In [8]:
#Stopword removal
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [9]:
df1["stwrd_removal_prob"]=df1["Problem_words"].apply(lambda x: [item for item in x if item not in stop])

In [10]:
df1["stwrd_removal_rect"]=df1["Rectification_words"].apply(lambda x: [item for item in x if item not in stop])

In [157]:
df1["Rectification_words"][0:1]

0    [CARRIED, OUT, DEFIBRILLATOR, BATTERY, INSPECT...
Name: Rectification_words, dtype: object

In [158]:
df1

,Work Card Ref,ATA,Problem,Rectification,Year,Unnamed: 5,Defect Type,Defect Item,Problem_words,Rectification_words,stwrd_removal_prob,stwrd_removal_rect
0,256404,25,DEFIBRILLATOR BATTERY - INSPECT/CHECK.,CARRIED OUT DEFIBRILLATOR BATTERY INSPECTION C...,2013,NaN,nan,nan,"[DEFIBRILLATOR, BATTERY, INSPECT, CHECK, ]","[CARRIED, OUT, DEFIBRILLATOR, BATTERY, INSPECT...","[DEFIBRILLATOR, BATTERY, INSPECT, CHECK, ]","[CARRIED, OUT, DEFIBRILLATOR, BATTERY, INSPECT..."
1,256407,25,DEFIBRILLATOR ELECTRODE PADS INSPECTION.,CARRIED OUT DEFIBRILLATOR ELECTRODE PADS INSPE...,2013,NaN,nan,nan,"[DEFIBRILLATOR, ELECTRODE, PADS, INSPECTION, ]","[CARRIED, OUT, DEFIBRILLATOR, ELECTRODE, PADS,...","[DEFIBRILLATOR, ELECTRODE, PADS, INSPECTION, ]","[CARRIED, OUT, DEFIBRILLATOR, ELECTRODE, PADS,..."
2,"324002, 324008, 324014, 324041",32,RH NOSE WHEEL TIRE FOUND WORN,RH NOSE WHEEL ASSEMBLY REMOVED AND SERVICEABLE...,2013,NaN,WORN,TIRE,"[RH, NOSE, WHEEL, TIRE, FOUND, WORN]","[RH, NOSE, WHEEL, ASSEMBLY, REMOVED, AND, SERV...","[RH, NOSE, WHEEL, TIRE, FOUND, WORN]","[RH, NOSE, WHEEL, ASSEMBLY, REMOVED, AND, SERV..."
3,NaN,24,TAIL COMPARTMENT GROUND SERVICE BUS INDICATOR ...,TAIL COMPARTMENT GROUND SERVICE BUS INDICATOR ...,2013,NaN,ILLUMINATED PERMANENTLY,INDICATOR,"[TAIL, COMPARTMENT, GROUND, SERVICE, BUS, INDI...","[TAIL, COMPARTMENT, GROUND, SERVICE, BUS, INDI...","[TAIL, COMPARTMENT, GROUND, SERVICE, BUS, INDI...","[TAIL, COMPARTMENT, GROUND, SERVICE, BUS, INDI..."
4,345004,34,VOR - IFR CHECK,CARRIED OUT VOR - IFR CHECK IN ACCORDANCE WITH...,2013,NaN,nan,nan,"[VOR, IFR, CHECK]","[CARRIED, OUT, VOR, IFR, CHECK, IN, ACCORDANCE...","[VOR, IFR, CHECK]","[CARRIED, OUT, VOR, IFR, CHECK, IN, ACCORDANCE..."
5,"308002, 308004",30,"PILOT REPORTS ""RH ICE DETECTOR AMBER MESSAGE A...",RH ICE DETECTOR PROBE REPLACED IN ACCORDANCE W...,2013,NaN,MESSAGE ALWAYS POSTED,ICE DETECTOR MESSAGE,"[PILOT, REPORTS, RH, ICE, DETECTOR, AMBER, MES...","[RH, ICE, DETECTOR, PROBE, REPLACED, IN, ACCOR...","[PILOT, REPORTS, RH, ICE, DETECTOR, AMBER, MES...","[RH, ICE, DETECTOR, PROBE, REPLACED, IN, ACCOR..."
6,NaN,34,TO CARRY OUT HONEYWELL ASB ADM/ADC/ADAHRS-34-AD1,NOT APPLICABLE - NOT AFFECTED BY SERIAL NO. IN...,2013,NaN,nan,nan,"[TO, CARRY, OUT, HONEYWELL, ASB, ADM, ADC, ADA...","[NOT, APPLICABLE, NOT, AFFECTED, BY, SERIAL, N...","[TO, CARRY, OUT, HONEYWELL, ASB, ADM, ADC, ADA...","[NOT, APPLICABLE, NOT, AFFECTED, BY, SERIAL, N..."
7,253201,25,ESPRESSO MAUER FOUND UNSERVICEABLE,DEFECTIVE ESPRESSO MAUER REMOVED AND SERVICEAB...,2013,NaN,UNSERVICEABLE,ESPRESSO MAUER,"[ESPRESSO, MAUER, FOUND, UNSERVICEABLE]","[DEFECTIVE, ESPRESSO, MAUER, REMOVED, AND, SER...","[ESPRESSO, MAUER, FOUND, UNSERVICEABLE]","[DEFECTIVE, ESPRESSO, MAUER, REMOVED, AND, SER..."
8,NaN,32,NOSE WHEEL ASSEMBLY TO BE BUILT UP.\nMEGGITT M...,"NOSE WHEEL ASSEMBLY DISASSEMBLED, CLEANED, INS...",2013,NaN,nan,nan,"[NOSE, WHEEL, ASSEMBLY, TO, BE, BUILT, UP, MEG...","[NOSE, WHEEL, ASSEMBLY, DISASSEMBLED, CLEANED,...","[NOSE, WHEEL, ASSEMBLY, TO, BE, BUILT, UP, MEG...","[NOSE, WHEEL, ASSEMBLY, DISASSEMBLED, CLEANED,..."
9,493402,49,FUEL FILTER BYPASS INDICATOR DETAILED INSPECTI...,FUEL FILTER BYPASS INDICATOR DETAILED INSPECTI...,2013,NaN,nan,nan,"[FUEL, FILTER, BYPASS, INDICATOR, DETAILED, IN...","[FUEL, FILTER, BYPASS, INDICATOR, DETAILED, IN...","[FUEL, FILTER, BYPASS, INDICATOR, DETAILED, IN...","[FUEL, FILTER, BYPASS, INDICATOR, DETAILED, IN..."


In [160]:
l=(len(df1))
for i in range(0,l): 
    j=len(df1["stwrd_removal_prob"][i])
    for k in range(0,j):
        if df1["Problem_words"][i][k]=='WORN':            
            df1["Defect Item"][i]="TIRE"
            df1["Defect Type"][i]="WORN"
        elif df1["Problem_words"][i][k]=='ILLUMINATED':            
             df1["Defect Item"][i]="INDICATOR"
             df1["Defect Type"][i]="ILLUMINATED PERMANENTLY"
        elif df1["Problem_words"][i][k]=='MESSAGE'and'ALWAYS':            
             df1["Defect Item"][i]="ICE DETECTOR MESSAGE"
             df1["Defect Type"][i]="MESSAGE ALWAYS POSTED" 
        elif df1["Problem_words"][i][k]=='WHEEL'and'BUILT':            
             df1["Defect Item"][i]="WHEEL ASSEMBLY"
             df1["Defect Type"][i]="BUILT UP"
        elif df1["Problem_words"][i][k]=='GEAR'and'LUBRICATION':            
             df1["Defect Item"][i]="GEAR"
             df1["Defect Type"][i]="LUBRICATION DUE"
        elif df1["Problem_words"][i][k]=='MAINTENANCE'and'PHASE':            
             df1["Defect Item"][i]="ENGINE"
             df1["Defect Type"][i]="MAINTAINANCE DUE"
        elif df1["Problem_words"][i][k]=='PAINT'and'FIRE':            
             df1["Defect Item"][i]="FIRE BOX"
             df1["Defect Type"][i]="PAINT DAMAGE"  
        elif df1["Problem_words"][i][k]=='NITROGEN'and'SERVED':            
             df1["Defect Item"][i]="WHEEL/TIRE"
             df1["Defect Type"][i]="NITROGEN"   
        elif df1["Problem_words"][i][k]=='INOPERATIVE'and'AMPLIFIER':            
             df1["Defect Item"][i]="AMPLIFIER"
             df1["Defect Type"][i]="INOPERATIVE"
        elif df1["Problem_words"][i][k]=='OPERATE'and'INTERMITTENTLY':            
             df1["Defect Item"][i]="TCAS"
             df1["Defect Type"][i]="OPERATE INTERMITTENTLY" 
        elif df1["Problem_words"][i][k]=='BATTERY':            
             df1["Defect Item"][i]="BATTERY"
             df1["Defect Type"][i]="FAIL" 
        elif df1["Problem_words"][i][k]=='SDU':
             df1["Defect Item"][i]="SDU"
             df1["Defect Type"][i]="FAIL"
        elif df1["Problem_words"][i][k]=='GPS':
             df1["Defect Item"][i]="GPS"
             df1["Defect Type"][i]="FAIL"
        elif df1["Problem_words"][i][k]=='LIGHTS':
             df1["Defect Item"][i]="LIGHTS"
             df1["Defect Type"][i]="UNSERVICEABLE"    
        elif df1["Problem_words"][i][k]=='HEATING':
             df1["Defect Item"][i]="HEATING SYSTEM"
             df1["Defect Type"][i]="UNSERVICEABLE"    
        elif df1["Problem_words"][i][k]=='SIGNS':
             df1["Defect Item"][i]="SIGNS BULBS"
             df1["Defect Type"][i]="NOT LIGHT UP"
        elif df1["Problem_words"][i][k]=='SIGNS':
             df1["Defect Item"][i]="SIGNS BULBS"
             df1["Defect Type"][i]="NOT LIGHT UP"
        elif df1["Problem_words"][i][k]=='OIL':
             df1["Defect Item"][i]="ENGINE"
             df1["Defect Type"][i]="OIL" 
        elif df1["Problem_words"][i][k]=='DISLOGED':
             df1["Defect Item"][i]="DOOR"
             df1["Defect Type"][i]="DISLOGED"    
        elif df1["Problem_words"][i][k]=='INOPERATIVE'and'AMPLIFIER':            
             df1["Defect Item"][i]="AMPLIFIER"
             df1["Defect Type"][i]="INOPERATIVE" 
        elif df1["Problem_words"][i][k]=='INOPERATIVE'and'LANDING'and'LIGHT':            
             df1["Defect Item"][i]="LANDING LIGHT"
             df1["Defect Type"][i]="INOPERATIVE"    
        elif df1["Problem_words"][i][k]=='ADS':            
             df1["Defect Item"][i]="ADS"
             df1["Defect Type"][i]="FAIL"    
            
            
            
    

/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWi

/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be s

In [164]:
df1["Defect Type"][0:50]

0                        FAIL
1                         nan
2                        WORN
3     ILLUMINATED PERMANENTLY
4                         nan
5       MESSAGE ALWAYS POSTED
6                         nan
7               UNSERVICEABLE
8                    BUILT UP
9                         nan
10                        nan
11                        nan
12                   NITROGEN
13                   NITROGEN
14                       WORN
15                       WORN
16                 RELOCATION
17                        nan
18                       FAIL
19                        nan
20                        nan
21                   BUILT UP
22                   BUILT UP
23                       WORN
24                       WORN
25                        nan
26                        nan
27                   BUILT UP
28                   BUILT UP
29                        nan
30                       FAIL
31                        nan
32                        nan
33     OPE

In [165]:
df1["Defect Item"][0:50]

0                  BATTERY
1                      nan
2                     TIRE
3                INDICATOR
4                      nan
5     ICE DETECTOR MESSAGE
6                      nan
7           ESPRESSO MAUER
8           WHEEL ASSEMBLY
9                      nan
10                     nan
11                     nan
12              WHEEL/TIRE
13              WHEEL/TIRE
14                    TIRE
15                    TIRE
16             FIRE BOTTLE
17                     nan
18                 BATTERY
19                     nan
20                     nan
21          WHEEL ASSEMBLY
22          WHEEL ASSEMBLY
23                    TIRE
24                    TIRE
25                     nan
26                     nan
27          WHEEL ASSEMBLY
28          WHEEL ASSEMBLY
29                     nan
30                 BATTERY
31                     nan
32                     nan
33                    TCAS
34          WHEEL ASSEMBLY
35              WHEEL/TIRE
36                    TIRE
3

In [166]:
df1["Problem_words"][0]

['DEFIBRILLATOR', 'BATTERY', 'INSPECT', 'CHECK', '']

In [167]:
l=(len(df1))
for i in range(0,l):     
    if 'MAIN' in df1["Problem_words"][i] :        
         print("Yes, 'MAIN' found in List : " , df1["Problem_words"][i])
else:
     print("not found")
    

Yes, 'MAIN' found in List :  ['NOSE', 'WHEEL', 'ASSEMBLY', 'TO', 'BE', 'BUILT', 'UP', 'MEGGITT', 'MAIN', 'WHEEL', 'ASSY', 'P', 'N', 'AH5505', 'GULFSTREAM', 'MAIN', 'WHEEL', 'ASSY', 'P', 'N', '1159SCL505', '7', 'S', 'NO', '5038KY']
Yes, 'MAIN' found in List :  ['ALL', 'MAIN', 'TIRES', 'TO', 'BE', 'SERVED', 'WITH', 'NITROGEN']
Yes, 'MAIN' found in List :  ['LH', 'INBOARD', 'MAIN', 'TIRE', 'FOUND', 'WORN', 'TO', 'LIMIT']
Yes, 'MAIN' found in List :  ['LH', 'OUTBOARD', 'MAIN', 'TIRE', 'FOUND', 'WORN', 'TO', 'LIMIT']
Yes, 'MAIN' found in List :  ['MAIN', 'WHEEL', 'ASSEMBLY', 'TO', 'BE', 'BUILT', 'UP', 'MEGGITT', 'P', 'N', 'AHA2114', 'MAIN', 'WHEEL', 'ASSY', 'P', 'N', '1159SCL503', '9', 'S', 'NO', '5090HX']
Yes, 'MAIN' found in List :  ['MAIN', 'WHEEL', 'ASSEMBLY', 'TO', 'BE', 'BUILT', 'UP', 'MEGGITT', 'P', 'N', 'AHA2114', 'MAIN', 'WHEEL', 'ASSY', 'P', 'N', '1159SCL503', '9', 'S', 'NO', '5086HX']
Yes, 'MAIN' found in List :  ['RH', 'INBOARD', 'MAIN', 'TIRE', 'FOUND', 'WORN', 'TO', 'LIMIT', '

In [43]:
# l=(len(df1))
# keyword_list = ['FOUND', 'ILLUMINATED' 'PERMANTENTLY','MESSAGE' 'ALWAYS' 'POSTED' ]
# Defect=[]
# for i in range(0,l):
#     if any(word in df1["Problem_words"][i] for word in keyword_list):
#         Defect.append('match')
#     else:
#         Defect.append('no match')
            

In [44]:
df1["stwrd_removal_prob"].isnull().values.any()

False

In [45]:
df1["Defect"]=Defect

In [46]:
df1["Defect"]

0      no match
1      no match
2         match
3      no match
4      no match
5      no match
6      no match
7         match
8      no match
9      no match
10     no match
11     no match
12     no match
13     no match
14        match
15        match
16     no match
17     no match
18     no match
19     no match
20     no match
21     no match
22     no match
23        match
24     no match
25     no match
26     no match
27     no match
28     no match
29     no match
         ...   
373    no match
374    no match
375    no match
376    no match
377    no match
378    no match
379    no match
380    no match
381    no match
382    no match
383    no match
384    no match
385    no match
386       match
387    no match
388    no match
389       match
390    no match
391    no match
392    no match
393    no match
394       match
395    no match
396    no match
397    no match
398    no match
399    no match
400    no match
401    no match
402    no match
Name: Defect, Length: 40

In [47]:
df1["Defect Type"][50:100]

50                       nan
51                       nan
52                       nan
53                       nan
54                       nan
55              PAINT DAMAGE
56              PAINT DAMAGE
57                  NITROGEN
58                  NITROGEN
59                       nan
60                       nan
61               INOPERATIVE
62                       nan
63                       nan
64                       nan
65                       nan
66                      WORN
67                       nan
68                       nan
69                       nan
70                       nan
71                       nan
72                  BUILT UP
73                  BUILT UP
74    OPERATE INTERMITTENTLY
75     MESSAGE ALWAYS POSTED
76                       nan
77                  BUILT UP
78                      WORN
79                  BUILT UP
80                  BUILT UP
81                      WORN
82                      WORN
83                      FAIL
84            

In [15]:
#create a pivot table
#df1.drop('Problem','Rectification','Unnamed: 5','Work Card Ref','Year',axis=1)
#df1.drop('Year',axis=1)
df1.pivot_table(index=['ATA ','Defect Item','Defect Type'], aggfunc='count')


Problem  Problem_words  \
ATA    Defect Item          Defect Type                                       
5      nan                  nan                           18             18   
8      nan                  nan                            2              2   
12     nan                  nan                            8              8   
21     nan                  nan                            6              6   
22     nan                  nan                            1              1   
23     nan                  nan                            5              5   
24     INDICATOR            ILLUMINATED PERMANENTLY        1              1   
       nan                  nan                           18             18   
25     ESPRESSO MAUER       UNSERVICEABLE                  1              1   
       nan                  nan                           57             57   
26     FIRE BOTTLE          RELOCATION                     1              1   
       nan                  nan                            2              2   
27     nan                  nan                            4              4   
28     nan                  nan                            2              2   
29     nan                  nan                            2              2   
30     ICE DETECTOR MESSAGE MESSAGE ALWAYS POSTED          1              1   
31     nan                  nan                            1              1   
32     TIRE                 LEAKING                        1              1   
                            NITROGEN                       2              2   
                            WORN                           5              5   
       WHEEL ASSEMBLY       BUILT UP                       4              4   
       nan                  nan                          121            121   
33     nan                  nan                            9              9   
34     TCAS                 OPERATE INTERMITTENTLY         1              1   
       nan                  nan                           42             42   
35     nan                  nan                            1              1   
38     nan                  nan                            3              3   
49     nan                  nan                           11             11   
52     nan                  nan                            3              3   
53     nan                  nan                            6              6   
55     nan                  nan                            1              1   
56     nan                  nan                            2              2   
71     nan                  nan                            4              4   
72     nan                  nan                           16             16   
73     nan                  nan                            5              5   
74     nan                  nan                            1              1   
79     nan                  nan                            1              1   
88     nan                  nan                            1              1   
98     nan                  nan                            1              1   
99     nan                  nan                            3              3   
05     nan                  nan                            6              6   
22\n76 nan                  nan                            1              1   
AE     nan                  nan                           17             17   

                                                     Rectification  \
ATA    Defect Item          Defect Type                              
5      nan                  nan                                 18   
8      nan                  nan                                  2   
12     nan                  nan                                  8   
21     nan                  nan                                  6   
22     nan                  nan                                  1   
23     

In [16]:
#writing data to excel

writer = ExcelWriter('output.xlsx') 
#wb  = writer.book
#out_path = "/Users/varun/Desktop/ST/"
#writer = pd.ExcelWriter(out_path , engine='xlsxwriter')

df1.to_excel(writer,sheet_name='Sheet1',index=False)   
writer.save() 
